In [1]:
# 準備工作

from bs4 import BeautifulSoup
# import requests
# import bs4
from selenium import webdriver
import time
import datetime
import re
import sqlite3
import os

db_name = "pacific.sqlite"
db_id_list = "pacific_IDs.sqlite"
target_url_list = list(range(71086,71206))

In [2]:
# create a database recording which IDs have been examined.

if os.path.isfile(db_id_list) == False:

    with sqlite3.connect(db_id_list) as conn_id_list:
        
        c = conn_id_list.cursor()
        
        c.execute("""CREATE TABLE idList(
        id text unique not null,
        done boolean not null default 0,
        empty boolean not null default 1,
        modDate datetime 
        )""")
    
    # Generate a list of potential IDs

    id_list = list(range(1,100000))
    id_list = [str(id).zfill(6) for id in id_list]
    id_list = id_list[::-1]

    today = datetime.datetime.today()
    today = today.strftime('%Y-%m-%d %H:%M:%S')

    with sqlite3.connect(db_id_list) as conn_id_list:    

        for eachID in id_list:
            command_string = """INSERT INTO idList(id, modDate) 
            values(""" + eachID + ",'" + str(today) + "');"
            conn_id_list.execute(command_string)

    conn_id_list.close()

In [3]:
# Create the database for storing data if it does not already exist.

if os.path.isfile(db_name) == False:

    with sqlite3.connect(db_name) as conn:
        
        c = conn.cursor()
        
        c.execute("""CREATE TABLE rental(
        url text unique not null,
        soup text not null,
        nosql text,
        rdb text,
        getTime datetime 
        )""")
        
    conn.close()

In [4]:
# Find all the URLs which have not been worked on before.
# Place all such URLs in id_list

with sqlite3.connect(db_id_list) as conn_done_url:
    
    c_du = conn_done_url.cursor()
    qryString = "SELECT id from idList where not done;"
    c_du.execute(qryString)
    id_list = c_du.fetchall()
    
conn_done_url.close()

In [5]:
# Find all the URLs which already exist in the main database.

with sqlite3.connect(db_name) as retr_url:
        
    c = retr_url.cursor()

    c.execute("SELECT url from rental")
    url_completed = c.fetchall()
    
retr_url.close()

In [6]:
# Eliminate IDs which are alreay in the main database from id_list
# for good measure

# for each_url in url_completed:
#     ID = each_url[0][-6:]
#     if ID in id_list:
#         id_list.remove(ID)

In [7]:
# Define method crawl()

# Examine each ID to see whether it is an empty record.
# If yes, then record only in the ID database.
# If no, then record in both the Id database and the
# main database.

def crawl(target_id):

    target_id = str(target_id)
    target_id = str(target_id).zfill(6)
    url = "https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=" + target_id

    # Call phantomJS

    driver = webdriver.PhantomJS()
    driver.get(url)
    time.sleep(2.5)
    page_empty = (driver.title == "太平洋房屋")
    columns_string = "url, soup, nosql, rdb, getTime"

    # Store the results if the page does not return an empty record.

    today = datetime.datetime.today()
    today = today.strftime('%Y-%m-%d %H:%M:%S')
    
    if not page_empty:

        soup = driver.page_source
        soup = soup.replace("\"","\"\"")    # Need to escape the slash...
        soup = soup.replace('\'',"\'\'")

        nosql = ""
        rdb = ""

        with sqlite3.connect(db_name) as conn3:

            c3 = conn3.cursor()

            values = "'" + url + "',\"\"\"" + soup + "\"\"\",'" + nosql + "','" + rdb + "','" + str(today) + "'"       

            insertString = "INSERT INTO rental (" + columns_string + ") VALUES(" + values + ");"
            c3.execute(insertString)
        
        conn3.close()
        
        print(str(today) + ": " + str(target_id) + " is not empty; crawled.",end='\r')
    
    else:
        print(str(today) + ": " + str(target_id) + " is empty; skipped.    ",end='\r')
            
    # Update db_id_list to show that the page has been worked on.   

    with sqlite3.connect(db_id_list) as conn_update_id_list:

        c_uil = conn_update_id_list.cursor()

        if page_empty:
            empty = 1
        else:
            empty = 0

        updateString = "UPDATE idList SET done = 1, empty = " + str(empty) + " , modDate = '" + today + "' WHERE id = " + target_id + ";"
        c_uil.execute(updateString)
    
    conn_update_id_list.close()

In [8]:
# =========
# main loop

# target_total_count = len(target_url_list)
# count = 0

for each in target_url_list:
    try:
        each = str(each)
        crawl(each)
    except Exception as e:
        print(str(each) + ": " + str(e) +"        ",end='\r')
#     finally:
#         count += 1
#         percentage =  str(round(100 * (count / target_total_count),2))
#         print(str(count) + " IDs done. (" + percentage + "%)",end='\r')
    
print("The main loop has terminated.")

The main loop has terminated.s not empty; crawled.se        


In [13]:
# 檢查結果

with sqlite3.connect(db_name) as conn4:
    
    c4 = conn4.cursor()
    qryString = "SELECT distinct url from rental order by url desc;"
    c4.execute(qryString)
    results = c4.fetchall()

with sqlite3.connect(db_id_list) as conn55:
    
    c55 = conn55.cursor()
    qryString = "SELECT count(id) from idList where done = 1 and empty = 0 order by id desc;"
    c55.execute(qryString)
    results2 = c55.fetchall()

c4.close()
c55.close()

In [14]:
print(len(results))
print(results[0])

2693
('https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=071205',)
